# PRODUCT DIMENSION

In [1]:
import pandas as pd
from connection_script import connect_databases

In [2]:
db_op, db_etl = connect_databases()

## EXTRACT 

In [3]:
dim_product_category =  pd.read_sql_query("SELECT Name FROM [Production].[ProductCategory];", db_op)
dim_product_subcategory =  pd.read_sql_query("SELECT Name, ProductCategoryID FROM [Production].[ProductSubCategory];", db_op)
dim_product =  pd.read_sql_query("""
    SELECT t1.*, t2.Name AS ModelName, t4.LargePhoto, v1.
    FROM [Production].[Product] AS t1
    INNER JOIN [Production].[ProductModel] AS t2
    ON t1.ProductModelID = t2.ProductModelID
    INNER JOIN [Production].[ProductProductPhoto] AS t3
    ON t1.ProductID = t3.ProductID;
    INNER JOIN [Production].[ProductPhoto] AS t4
    ON t3.ProductPhotoID = t4.ProductPhotoID;
    INNER JOIN [Production].[vProductAndDescription] AS v1
    ON t1.ProductID = v1.ProductID
""", db_op)

## TRANSFORM

### Product Category

In [4]:
dim_product_category = dim_product_category.rename(columns={"Name":"EnglishProductCategoryName"})
dim_product_category = dim_product_category.assign(ProductCategoryAlternateKey=list(dim_product_category.index))
dim_product_category = dim_product_category[["ProductCategoryAlternateKey", "EnglishProductCategoryName"]]
dim_product_category

,ProductCategoryAlternateKey,EnglishProductCategoryName
0,0,Accessories
1,1,Bikes
2,2,Clothing
3,3,Components


### Product Subcategory

In [5]:
dim_product_subcategory = dim_product_subcategory.rename(columns={"Name":"EnglishProductSubcategoryName", "ProductCategoryID":"ProductCategoryKey"})
dim_product_subcategory = dim_product_subcategory.assign(ProductSubcategoryAlternateKey=list(dim_product_subcategory.index))
dim_product_subcategory = dim_product_subcategory[["ProductSubcategoryAlternateKey", "EnglishProductSubcategoryName", "ProductCategoryKey"]]
dim_product_subcategory

,ProductSubcategoryAlternateKey,EnglishProductSubcategoryName,ProductCategoryKey
0,0,Mountain Bikes,1
1,1,Road Bikes,1
2,2,Touring Bikes,1
3,3,Handlebars,2
4,4,Bottom Brackets,2
5,5,Brakes,2
6,6,Chains,2
7,7,Cranksets,2
8,8,Derailleurs,2
9,9,Forks,2


### Product

In [ ]:
dim_product = dim_product[["ProductNumber",
                           "ProductSubcategoryID",
                           "WeightUnitMeasure",
                           "SizeUnitMeasure",
                           "Name",
                           "StandardCost",
                          "FinishedGoodsFlag",
                          "Color",
                           "SafetyStockLevel",
                           "ReorderPoint",
                           "ListPrice",
                           "Size",
                           ##"SizeRange" missing
                           "Weight",
                           "DaysToManufacture",
                           "ProductLine",
                           "Class",
                           "Style",
                           "ModelName",
                           "LargePhoto"
                          ]]
dim_product = dim_product.rename(columns={"ProductNumber":"ProductAlternateKey",
                                          "ProductSubcategoryID":"ProductSubcategoryKey",
                                          "Name":"EnglishProductName"
                                         })